In [ ]:
# Bài thực hành 1: Nhận dạng chữ số viết tay với MNIST (Chi tiết cho người mới)
# ====================================================================================

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# tensorflow: Framework học máy và deep learning
import tensorflow as tf
# Tải bộ dữ liệu MNIST từ keras (là một phần của tensorflow)
from tensorflow.keras.datasets import mnist
# Sequential: Kiểu mô hình neural network đơn giản nhất trong keras, các layer xếp tuần tự
from tensorflow.keras.models import Sequential
# Dense: Layer kết nối đầy đủ, Flatten: Layer làm phẳng dữ liệu
from tensorflow.keras.layers import Dense, Flatten
# to_categorical: Hàm chuyển đổi nhãn thành biểu diễn one-hot
from tensorflow.keras.utils import to_categorical

In [ ]:
# -------------------------------------------------------------
# STEP 1: TẢI DỮ LIỆU MNIST
# -------------------------------------------------------------
print("Step 1: Tải dữ liệu MNIST")

In [ ]:
# mnist.load_data(): Hàm tải bộ dữ liệu MNIST
# Hàm này trả về hai tuple: (X_train, y_train) và (X_test, y_test)
# X_train: Dữ liệu hình ảnh huấn luyện, y_train: Nhãn tương ứng
# X_test: Dữ liệu hình ảnh kiểm tra, y_test: Nhãn tương ứng
(X_train, y_train), (X_test, y_test)= mnist.load_data()

In [ ]:
# In thông tin về kích thước dữ liệu
# X_train.shape: Kích thước của mảng X_train
# Kết quả có dạng (60000, 28, 28) nghĩa là 60000 hình ảnh, mỗi hình 28x28 pixel
print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")

In [ ]:
# In số lượng mẫu trong mỗi tập
# X_train.shape[0]: Lấy kích thước của chiều đầu tiên (số lượng mẫu)
print(f"Số lượng mẫu huấn luyện: {X_train.shape[0]}")
print(f"Số lượng mẫu kiểm tra: {X_test.shape[0]}")

In [ ]:
# In kích thước mỗi hình ảnh
print(f"Kích thước mỗi hình ảnh: {X_train.shape[1]}x{X_train.shape[2]} pixels")

In [ ]:
# -------------------------------------------------------------
# STEP 2: HIỂN THỊ MỘT SỐ ẢNH VÍ DỤ
# -------------------------------------------------------------
print("\nStep 2: Hiển thị một số ví dụ từ tập huấn luyện")

# Chọn một chỉ số cụ thể (thứ 150) để hiển thị
idx = 150

# plt.figure(): Tạo một figure mới với kích thước 6x3 inch
plt.figure(figsize=(6, 3))
# plt.subplot(1, 2, 1): Chia figure thành 1 hàng, 2 cột, và chọn vị trí 1
plt.subplot(1, 2, 1)
# plt.imshow(): Hiển thị hình ảnh
# X_train[idx]: Lấy hình ảnh thứ idx từ tập huấn luyện
# cmap=plt.cm.gray: Hiển thị ở chế độ grayscale
plt.imshow(X_train[idx], cmap=plt.cm.gray)
# plt.title(): Thêm tiêu đề cho hình ảnh
# f-string cho phép nhúng biểu thức Python trực tiếp vào chuỗi bằng cú pháp {expression}
plt.title(f'Corresponding label: {y_train[idx]}\n% {y_train[idx]}', fontsize=20)

# Hiển thị thêm 5 ví dụ
# Tạo figure mới với kích thước 12x4 inch
plt.figure(figsize=(12, 4))
# Dùng vòng lặp for để hiển thị 5 hình ảnh
for i in range(5):
    # Chia figure thành 1 hàng, 5 cột và chọn vị trí i+1
    plt.subplot(1, 5, i+1)
    # Hiển thị hình ảnh thứ i trong tập huấn luyện
    plt.imshow(X_train[i], cmap='gray')
    # Thêm tiêu đề với nhãn tương ứng
    plt.title(f"Label: {y_train[i]}")
    # Tắt trục (không hiển thị trục x, y)
    plt.axis('off')
# Tự động điều chỉnh layout cho đẹp
plt.tight_layout()
# Hiển thị hình ảnh
plt.show()

In [ ]:
# -------------------------------------------------------------
# STEP 3: TIỀN XỬ LÝ DỮ LIỆU
# -------------------------------------------------------------
print("\nStep 3: Tiền xử lý dữ liệu")

# Chuyển đổi kiểu dữ liệu sang float32
# Ban đầu dữ liệu có kiểu uint8 (0-255), chuyển sang float32 để tính toán
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
# Chuẩn hóa dữ liệu bằng cách chia cho 255
# Giá trị pixel ban đầu từ 0-255, chia cho 255 để chuyển về khoảng [0,1]
# Điều này giúp quá trình huấn luyện ổn định và hội tụ nhanh hơn
X_train /= 255
X_test /= 255

In [ ]:
# In thông tin về giá trị pixel trước và sau khi chuẩn hóa
print(f"Giá trị pixel tối đa trước khi chuẩn hóa: 255")
print(f"Giá trị pixel tối đa sau khi chuẩn hóa: {X_train.max()}")
print(f"Giá trị pixel tối thiểu sau khi chuẩn hóa: {X_train.min()}")

In [ ]:
# Định dạng lại dữ liệu: chuyển từ hình ảnh 28x28 sang vector 784
# reshape(): Hàm thay đổi kích thước mảng mà không thay đổi dữ liệu
# X_train.shape[0]: Số lượng mẫu
# 784 = 28*28: Số pixel trong mỗi hình ảnh sau khi làm phẳng
X_train_flatten = X_train.reshape(X_train.shape[0], 784)
X_test_flatten = X_test.reshape(X_test.shape[0], 784)

In [ ]:
# One-hot encoding cho nhãn
# Chuyển đổi nhãn từ dạng số (0-9) sang dạng vector one-hot
# Ví dụ: 5 -> [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
# num_classes=10: Có 10 lớp (0-9)
y_train_categorical = to_categorical(y_train, num_classes=10)
y_test_categorical = to_categorical(y_test, num_classes=10)

In [ ]:
# Hiển thị thông tin sau khi tiền xử lý
print(f"Kích thước dữ liệu huấn luyện sau khi làm phẳng: {X_train_flatten.shape}")
print(f"Kích thước dữ liệu kiểm tra sau khi làm phẳng: {X_test_flatten.shape}")
print(f"Hình dạng nhãn huấn luyện sau khi one-hot encoding: {y_train_categorical.shape}")
print(f"Ví dụ one-hot encoding cho chữ số {y_train[0]}: {y_train_categorical[0]}")

In [ ]:
# -------------------------------------------------------------
# STEP 4: THIẾT KẾ VÀ HUẤN LUYỆN MẠNG NƠ-RON
# -------------------------------------------------------------
print("\nStep 4: Thiết kế và huấn luyện mạng nơ-ron")

In [ ]:
# Xây dựng mô hình Sequential (các layer xếp tuần tự)
model = Sequential([
    # Lớp Flatten để chuyển đổi hình ảnh 28x28 thành vector 784
    # input_shape=(28, 28): Kích thước đầu vào là hình ảnh 28x28
    Flatten(input_shape=(28, 28)),

    # Lớp Dense (kết nối đầy đủ) với 128 nơ-ron
    # activation='relu': Hàm kích hoạt ReLU (Rectified Linear Unit)
    # ReLU trả về max(0, x), giúp mô hình học các mối quan hệ phi tuyến
    Dense(128, activation='relu'),

    # Lớp đầu ra với 10 nơ-ron (cho 10 chữ số 0-9)
    # activation='softmax': Hàm kích hoạt softmax biến đầu ra thành phân phối xác suất
    # (tổng xác suất các lớp bằng 1)
    Dense(10, activation='softmax')
])


# Hiển thị tóm tắt thông tin mô hình
# Hiển thị số lượng tham số (weights và biases) của mỗi layer và tổng số
model.summary()


In [ ]:
# Biên dịch mô hình
# optimizer='adam': Thuật toán tối ưu Adam, phổ biến và hiệu quả
# loss='categorical_crossentropy': Hàm mất mát cho bài toán phân loại nhiều lớp
# metrics=['accuracy']: Theo dõi độ chính xác trong quá trình huấn luyện
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Huấn luyện mô hình
print("\nBắt đầu huấn luyện mô hình...")
# model.fit(): Hàm huấn luyện mô hình
# X_train, y_train_categorical: Dữ liệu huấn luyện và nhãn
# epochs=10: Số lần lặp qua toàn bộ dữ liệu huấn luyện
# batch_size=32: Số mẫu được xử lý trong mỗi bước cập nhật
# validation_split=0.1: Dùng 10% dữ liệu huấn luyện làm tập validation
# verbose=1: Hiển thị thanh tiến trình trong quá trình huấn luyện
history = model.fit(
    X_train, y_train_categorical,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

In [ ]:
# -------------------------------------------------------------
# STEP 5: ĐÁNH GIÁ MÔ HÌNH
# -------------------------------------------------------------
print("\nStep 5: Đánh giá mô hình")

# Đánh giá mô hình trên tập kiểm tra
# model.evaluate(): Tính toán loss và các metrics trên dữ liệu kiểm tra
# verbose=0: Không hiển thị thanh tiến trình
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Vẽ đồ thị độ chính xác và mất mát trong quá trình huấn luyện
plt.figure(figsize=(12, 4))

# Đồ thị độ chính xác (accuracy)
plt.subplot(1, 2, 1)
# history.history['accuracy']: Độ chính xác trên tập huấn luyện
plt.plot(history.history['accuracy'], label='Training Accuracy')
# history.history['val_accuracy']: Độ chính xác trên tập validation
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Đồ thị mất mát (loss)
plt.subplot(1, 2, 2)
# history.history['loss']: Mất mát trên tập huấn luyện
plt.plot(history.history['loss'], label='Training Loss')
# history.history['val_loss']: Mất mát trên tập validation
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# -------------------------------------------------------------
# STEP 6: DỰ ĐOÁN VÀ TRỰC QUAN HÓA KẾT QUẢ
# -------------------------------------------------------------
print("\nStep 6: Dự đoán và trực quan hóa kết quả")

# Dự đoán trên 10 ảnh đầu tiên từ tập kiểm tra
# model.predict(): Dự đoán đầu ra cho dữ liệu đầu vào
predictions = model.predict(X_test[:10])

# np.argmax(predictions, axis=1): Tìm chỉ số có giá trị lớn nhất trên mỗi hàng
# axis=1 nghĩa là tìm giá trị lớn nhất theo chiều ngang (trên mỗi hàng)
# Đây chính là chữ số được dự đoán
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế của 10 ảnh đầu tiên
true_classes = y_test[:10]

# Hiển thị hình ảnh và kết quả dự đoán
plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(X_test[i], cmap='gray')

    # Chọn màu dựa trên dự đoán đúng/sai
    # Nếu dự đoán đúng: màu xanh, nếu sai: màu đỏ
    color = 'green' if predicted_classes[i] == true_classes[i] else 'red'

    plt.title(f"Thực tế: {true_classes[i]}\nDự đoán: {predicted_classes[i]}",
             color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# -------------------------------------------------------------
# STEP 7: TẠO MA TRẬN NHẦM LẪN (CONFUSION MATRIX)
# -------------------------------------------------------------
# Import các công cụ cần thiết từ scikit-learn
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Dự đoán trên toàn bộ tập kiểm tra
y_pred = model.predict(X_test)
# Chuyển từ dạng one-hot về dạng số
y_pred_classes = np.argmax(y_pred, axis=1)

# Tạo ma trận nhầm lẫn
# Ma trận nhầm lẫn cho biết số lượng mẫu của lớp thực tế i được dự đoán là lớp j
cm = confusion_matrix(y_test, y_pred_classes)

# Hiển thị ma trận nhầm lẫn
plt.figure(figsize=(10, 8))
# sns.heatmap(): Tạo biểu đồ heatmap
# annot=True: Hiển thị giá trị trong mỗi ô
# fmt="d": Định dạng giá trị là số nguyên
# cmap="Blues": Sử dụng bảng màu xanh dương
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title('Ma trận nhầm lẫn')
plt.show()

In [ ]:
# -------------------------------------------------------------
# STEP 8: LƯU MÔ HÌNH (TÙY CHỌN)
# -------------------------------------------------------------
# Lưu mô hình để có thể sử dụng lại sau này
# model.save(): Lưu toàn bộ mô hình (kiến trúc, trọng số, cấu hình)
model.save('mnist_model.h5')
print("\nĐã lưu mô hình vào tệp 'mnist_model.h5'")

In [ ]:
# -------------------------------------------------------------
# STEP 9: KIỂM TRA MÔ HÌNH VỚI MỘT ẢNH TÙY Ý
# -------------------------------------------------------------
print("\nStep 9: Kiểm tra mô hình với một ảnh tùy ý")

# Chọn một ảnh ngẫu nhiên từ tập kiểm tra
import random
random_idx = random.randint(0, len(X_test)-1)
test_image = X_test[random_idx]

# Hiển thị ảnh
plt.figure(figsize=(4, 4))
plt.imshow(test_image, cmap='gray')
plt.title(f"Nhãn thực tế: {y_test[random_idx]}")
plt.axis('off')
plt.show()

# Dự đoán
# Cần reshape ảnh thành batch size 1 để phù hợp với đầu vào mô hình
# np.expand_dims(): Thêm một chiều vào mảng
single_prediction = model.predict(np.expand_dims(test_image, axis=0))
# Lấy chỉ số của lớp có xác suất cao nhất
predicted_class = np.argmax(single_prediction)

print(f"Ảnh thực tế là chữ số: {y_test[random_idx]}")
print(f"Mô hình dự đoán là chữ số: {predicted_class}")
print(f"Xác suất dự đoán cho mỗi chữ số:")
for i in range(10):
    print(f"  Chữ số {i}: {single_prediction[0][i]*100:.2f}%")